# Valor de mercado de todas as companhias listadas na bolsa

## Bibliotecas importadas

In [1]:
# Bibliotecas importadas
from datetime import datetime
from time import sleep
import pandas as pd
import requests
import random
import sqlite3
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
options = Options()
options.binary_location = r'C:\Program Files (x86)\Mozilla Firefox\firefox.exe' # Localicação do programa firefox.exe

In [2]:
driver = webdriver.Firefox(executable_path='../webdrivers/geckodriver.exe', options=options) # Localicação do webdriver

C:\Users\UFT\AppData\Local\Temp\ipykernel_2428\2149489876.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path='../webdrivers/geckodriver.exe', options=options) # Localicação do webdriver


## Obtenção do valor de mercado das companhias listadas

> O valor de mercado (também chamado de market cap) de uma companhia listada em bolsa é calculado multiplicando a base acionária – ou seja, a quantidade de ações da empresa – pelo preço individual de cada classe de ações (https://www.b3.com.br/pt_br/noticias/valor-de-mercado.htm).

![Valor de Mercado das empresas listadas na B3](img/b3-valor-de-mercado.png)

> O valor de mercado das empresas listadas é obtido pelo valor da última cotação disponível da ação, multiplicado pela quantidade de ações existentes, e deve ser observado em conjunto com outras variáveis, como liquidez, volume, negociabilidade da espécie da ação.
Dessa forma, os valores disponibilizados representam tão-somente estimativas genéricas que seguem critérios pré-estabelecidos e podem não corresponder ao valor real de mercado (https://www.b3.com.br/pt_br/market-data-e-indices/servicos-de-dados/market-data/consultas/mercado-a-vista/valor-de-mercado-das-empresas-listadas/bolsa-de-valores-mensal/).

Para esta pesquisa, será considerado o valor de mercado mensal, disponível em https://www.b3.com.br/pt_br/noticias/valor-de-mercado.htm, e datado de 04/03/2022.

In [3]:
valor_mercado = pd.read_csv('data/Bolsa_Valores_Mensal.csv', delimiter='|', skiprows=1, skipfooter=3, engine='python')

In [4]:
valor_mercado = valor_mercado[['IBOV', 'IBRX100', 'Empresa', 'Valor (R$ Mil) em 31/01/2022']]

In [5]:
valor_mercado.head()

,IBOV,IBRX100,Empresa,Valor (R$ Mil) em 31/01/2022
0,*,**,3R PETROLEUM,7.747159e+09
1,NaN,NaN,3TENTOS,4.842353e+09
2,NaN,NaN,ABC BRASIL,1.784291e+09
3,NaN,NaN,ACO ALTONA,1.953900e+08
4,NaN,NaN,AERIS,4.942077e+09


In [6]:
valor_mercado.tail()

,IBOV,IBRX100,Empresa,Valor (R$ Mil) em 31/01/2022
391,NaN,NaN,WILSON SONS,4.417499e+09
392,NaN,NaN,WIZ S.A.,1.312839e+09
393,NaN,NaN,WLM IND COM,1.171560e+09
394,NaN,NaN,XP INC,5.923644e+09
395,*,**,YDUQS PART,6.833954e+09


In [7]:
empresas = valor_mercado['Empresa'].to_list()
empresas[:5]

['3R PETROLEUM', '3TENTOS', 'ABC BRASIL', 'ACO ALTONA', 'AERIS']

In [8]:
empresa = valor_mercado.iloc[13].to_dict()

In [9]:
empresa

{'IBOV': nan,
 'IBRX100': nan,
 'Empresa': 'ALFA INVEST',
 'Valor (R$ Mil) em 31/01/2022': 637196316.35}

In [10]:
url = 'https://www.b3.com.br/pt_br/produtos-e-servicos/negociacao/renda-variavel/empresas-listadas.htm'

In [11]:
driver.get(url)

In [12]:
def obtem_url_empresa(driver, empresa):
    # URL da página de busca por empresas
    url_base = "https://sistemaswebb3-listados.b3.com.br/listedCompaniesPage/"
    
    # Elementos XPath
    input_company_name_xp = '//*[@id="keyword"]'
    btn_busca_xp = '//*[@id="accordionName"]/div/app-companies-home-filter-name/form/div/div[3]/button' # //*[@id="accordionName"]/div/app-companies-home-filter-name/form/div/div[3]/button
    card_empresa_xp = '//*[@id="nav-bloco"]/div/div/div/div'
    
    try:
        driver.get(url_base)
        #time_sleep()
        sleep(1)
        driver.find_element(By.XPATH, input_company_name_xp).send_keys(empresa)

        driver.find_element(By.XPATH, btn_busca_xp).click()

        #time_sleep()
        sleep(1)
        driver.find_element(By.XPATH, card_empresa_xp).click()

        #time_sleep()
        sleep(1)
        url_empresa = driver.current_url
    except:
        url_empresa = ''
    return url_empresa

In [13]:
url_empresa = obtem_url_empresa(driver, 'ALFA INVEST')
url_empresa

'https://sistemaswebb3-listados.b3.com.br/listedCompaniesPage/main/1384/BRIV/overview?language=pt-br'

In [14]:
def captura_conteudo(soup, title, tag_title, tag_content):
    try:
        element = soup.find(lambda tag: tag.name == tag_title and title in tag.text)
        content = element.find_next(tag_content).text.strip().replace('\n', '')
        for i in range(1,10):
            content = content.replace(i * ' ', ' ')
    except:
        content = ''
    return content

In [15]:
def split_classificacao_setorial(class_setorial):
    setorial_list_keys = [
        'Setor Econômico',
        'Subsetor Econômico',
        'Segmento Econômico',
    ]
    setorial_list_values = [_.strip() for _ in class_setorial.split('/')]
    dic = dict(zip(setorial_list_keys, setorial_list_values))
    return dic

In [85]:
def obtem_dados_empresa(driver, empresa_dict):
    d = empresa_dict
    url_empresa = obtem_url_empresa(driver, d['Empresa'])
    sleep(1)
    html = driver.page_source
    sleep(1)
    soup = BeautifulSoup(html)
    cnpj = captura_conteudo(soup, 'CNPJ', 'p', 'p')
    cod_cvm = captura_conteudo(soup, 'Código CVM', 'b', 'p')
    atividade_principal = captura_conteudo(soup, 'Atividade Principal', 'p', 'p')
    class_setorial = captura_conteudo(soup, 'Classificação Setorial', 'p', 'p')
    site = captura_conteudo(soup, 'Site', 'p', 'p')
    den_social = soup.find('h2').text
    cod_negociacao = captura_conteudo(soup, 'Código de Negociação', 'p', 'p')
    dict_setorial = split_classificacao_setorial(class_setorial)
    d.update({
        'CNPJ': cnpj,
        'Código CVM': cod_cvm,
        'Código de Negociação': cod_negociacao,
        'Denominação Social': den_social,
        'Atividade Principal': atividade_principal
    })
    d.update(dict_setorial)
    d.update({'Site': site, 'Url Fonte': url_empresa})
    return d

In [103]:
obtem_dados_empresa(driver, valor_mercado.iloc[0].to_dict())

{'IBOV': '*',
 'IBRX100': '**',
 'Empresa': '3R PETROLEUM',
 'Valor (R$ Mil) em 31/01/2022': 7747159302.72,
 'CNPJ': '12.091.809/0001-55',
 'Código CVM': '25291',
 'Código de Negociação': 'RRRP3',
 'Denominação Social': '3R PETROLEUM ÓLEO E GÁS S.A',
 'Atividade Principal': 'Exploração e produção de petróleo e gás natural.',
 'Setor Econômico': 'Petróleo. Gás e Biocombustíveis',
 'Subsetor Econômico': 'Petróleo. Gás e Biocombustíveis',
 'Segmento Econômico': 'Exploração. Refino e Distribuição',
 'Site': '',
 'Url Fonte': 'https://sistemaswebb3-listados.b3.com.br/listedCompaniesPage/main/25291/RRRP/overview?language=pt-br'}

In [104]:
card_body = driver.find_element(By.CLASS_NAME, 'card-body')

In [105]:
tbl = card_body.find_element(By.TAG_NAME, 'table')

In [106]:
tbl

<selenium.webdriver.remote.webelement.WebElement (session="b3fc2863-c4bb-41b4-ba1b-634c969074dc", element="7aeb4dbd-efb2-43e8-95c3-4ee5eb0a3583")>

In [107]:
tds = tbl.find_elements(By.TAG_NAME, 'td')

In [108]:
for td in tds[2:-1:2]:
    print(td.text)